In [1]:
from autogen import ConversableAgent, LLMConfig, run
from autogen.agentchat.groupchat.groupchat import GroupChatManager
from autogen.events.agent_events import InputRequestEvent
from autogen.io.event_processors import ConsoleEventProcessor
from autogen.tools import tool

llm_config = LLMConfig.from_json(path="OAI_CONFIG_LIST").where(model=["gpt-4o-mini"], api_type="openai")

llm_config

LLMConfig(config_list=[{'api_type': 'openai', 'model': 'gpt-4o-mini', 'api_key': '**********', 'tags': ['gpt-4o-mini', 'tool', 'vision']}])

# Single agent run

In [ ]:
from autogen.run_patterns.round_robin import RoundRobinRunPattern


@tool()
def get_meaning_of_life():
    return "The meaning of life is 42."


with llm_config:
    agent = ConversableAgent(name="Alice")

get_meaning_of_life.register_for_llm(agent)

response = run(
    agent, # or receiver=agent
    message="What is the meaning of life?", # or message="What is the meaning of life?"
    terminate_on=MaxTurns(4) or Keyword("TERMINATE"),  # implement arithmetics for this
    # default values for the following arguments:
    # sender="user",  # if not specified, we assume the sender is the user
    # run_pattern=RoundRobinRunPattern(),  # default run pattern is round robin
)


for event in response.events:
    print(f"Got event: {event}")
    if isinstance(event, InputRequestEvent):
        event.content.respond(input(event.content.prompt))

print(response.summary)

Got event: type='text' content=TextEvent(uuid=UUID('a50e62a6-7d7e-4e5c-8b26-32552c0f76f5'), content='What is the meaning of life?', sender_name='Alice', recipient_name='chat_manager')
Got event: type='group_chat_run_chat' content=GroupChatRunChatEvent(uuid=UUID('47673fe8-ae21-4d8b-b24a-c146f9089b78'), speaker_name='Alice', verbose=True)
Got event: type='using_auto_reply' content=UsingAutoReplyEvent(uuid=UUID('1031b1d3-9175-4446-8e37-023393edb415'), human_input_mode='TERMINATE', sender_name='chat_manager', recipient_name='Alice')
Got event: type='tool_call' content=ToolCallEvent(uuid=UUID('e08a5090-edb2-4d14-9968-7b3ccc104ca8'), content=None, sender_name='Alice', recipient_name='chat_manager', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ToolCall(id='call_ZtAx2L5lYgTt9inLFtb80GVw', function=FunctionCall(name='get_meaning_of_life', arguments='{}'), type='function')])
Got event: type='group_chat_run_chat' content=GroupChatRunChatEvent(uuid=UUID('734dd816-2e5

## With console processor

In [ ]:
from autogen import UserProxyAgent


@tool()
def get_meaning_of_life():
    return "The meaning of life is 42."


with llm_config:
    agent = ConversableAgent(name="Alice")

user_proxy = UserProxyAgent("user_proxy")

get_meaning_of_life.register_for_llm(agent)

response = run(sender=user_proxy, receiver=agent, message="What is the meaning of life?")

processor = ConsoleEventProcessor()
processor.process(response)

print(response.summary)

# Two agent run

In [ ]:
with llm_config:
    agent_1 = ConversableAgent(name="Jane")
    agent_2 = ConversableAgent(name="Bob")

response = run(sender=agent_1, receiver=agent_2, message="have a joke off", terminate_on=MaxTurns(5))


for event in response.events:
    print(f"Got event: {event.type=}, {event=}")
    if isinstance(event, InputRequestEvent):
        event.content.respond(input(event.prompt))

print(response.summary)

## With event processor

In [ ]:
with llm_config:
    agent_1 = ConversableAgent(name="Jane")
    agent_2 = ConversableAgent(name="Bob")

response = run(sender=agent_1, receiver=agent_2, message="have a joke off", terminate_on=Keyword("TERMINATE"))


ConsoleEventProcessor().process(response)

print(response.summary)

# Multi agent run with chat manager

In [ ]:
from autogen.run_patterns import GroupRunPattern


@tool()
def submit_plan(plan: str) -> str:
    return f"Plan submitted: {plan}"


with llm_config:
    planner = ConversableAgent(
        name="Planner",
        system_message="You are a planner. Collaborate with teacher and reviewer to create lesson plans.",
    )

    reviewer = ConversableAgent(
        name="Reviewer",
        system_message="You are a reviewer. Review lesson plans against 4th grade curriculum. Provide max 3 changes.",
    )

    teacher = ConversableAgent(
        name="Teacher",
        system_message="You are a teacher. Choose topics and work with planner and reviewer. Say DONE! when finished.",
    )
    run_pattern = GroupRunPattern(planner, reviewer, teacher)

submit_plan.register_for_llm(teacher)

response = run(
    receiver=planner,
    message="Create lesson plans for 4th grade.",
    run_pattern=run_pattern,
    terminate_on=Keyword("DONE!") or MaxTurns(10),
)

for event in response.events:
    print(f"Got event: {event.type=}, {event=}")
    if isinstance(event, InputRequestEvent):
        event.content.respond(input(event.prompt))

print(response.summary)

# Swarm chat manager

In [ ]:
from autogen import AfterWorkOption, ConversableAgent
from autogen.run_patterns import SwarmRunPattern

planner_message = """You are a classroom lesson planner.
Given a topic, write a lesson plan for a fourth grade class.
If you are given revision feedback, update your lesson plan and record it.
Use the following format:
<title>Lesson plan title</title>
<learning_objectives>Key learning objectives</learning_objectives>
<script>How to introduce the topic to the kids</script>
"""

reviewer_message = """You are a classroom lesson reviewer.
You compare the lesson plan to the fourth grade curriculum
and provide a maximum of 3 recommended changes for each review.
Make sure you provide recommendations each time the plan is updated.
"""

teacher_message = """You are a classroom teacher.
You decide topics for lessons and work with a lesson planner.
and reviewer to create and finalise lesson plans.
"""

with llm_config:
    # 1. Create our agents

    planner = ConversableAgent(name="planner_agent", system_message=planner_message)
    reviewer = ConversableAgent(name="reviewer_agent", system_message=reviewer_message)
    teacher = ConversableAgent(name="teacher_agent", system_message=teacher_message)

    run_pattern = SwarmRunPattern(after_work=AfterWorkOption.SWARM_MANAGER)

response = run(
    planner,
    message="Today, let's introduce our kids to the solar system.",
    run_pattern=run_pattern,
    terminate_on=MaxTurns(10),
)

for event in response.events:
    print(f"Got event: {event.type=}, {event=}")
    if isinstance(event, InputRequestEvent):
        event.content.respond(input(event.prompt))

print(response.summary)

# Sequential chat

# Nested Chat